<a href="https://colab.research.google.com/github/adl09/GPU_IB/blob/main/Clase1/ICNPG_HolaMundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entorno de ejecución

Google colab tiene varios entornos de ejecución con distintos recursos asociados. Para usar GPUs gratuitamente, podemos elegir el entorno T4 GPU, seleccionandolo en "Entorno de Ejecución". Por default solo podemos usar CPU.

Chequiemos luego rápidamente que tenemos acceso a una GPU de nvidia, con el siguiente comando:

In [31]:
!nvidia-smi

Mon Feb 10 13:15:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

el ! antes de nvidia-smi es para indicar que es un comando de linux lo que queremos correr (sino, por default todo es python).  

# Hola Mundo

En colab podemos escribir directamente codigo en python, pero también en muchos otros lenguajes no interpretados, en particular C y C++.

El truco para usar colab con códigos compilables es la primer linea %%writefile para indicar que queremos que se genere un archivo con el contenido de la celda, para luego compilarlo con alguno de los compiladores preinstalados en colab.

In [28]:
%%writefile hola.cu

/*
 * Imprime "hola mundo" desde 10 hilos corriendo en GPU
 */


#include <stdio.h>

__device__ void helloFromDevice()  // <-- las funciones device son aquellas que pueden llamarse desde un kernel
{
    printf("hola mundo desde el device, hilo: %d, bloque: %d\n",threadIdx.x,blockIdx.x);
}

__global__ void helloFromGPU() // <-- kernel (solo puede ser llamada desde el host)
{
    // esto lo hace cada hilo de la grilla
    printf("hola mundo desde la GPU!, hilo: %d, bloque: %d\n",threadIdx.x,blockIdx.x);
    helloFromDevice();

}

//int main(int argc, char **argv)
int main()
{
    // lanzamiento del kernel con una grilla de 1 bloque y 10 hilos por bloque
    // es asincronico respecto de del hilo de host
    helloFromGPU<<<2, 10>>>(); // (numero de bloques, numero de hilos por bloque)  <- conf. de la grilla

    // esto lo hace la CPU independientemente de si el kernel termino o no...
    printf("Hola mundo desde la CPU!\n\n");

    helloFromGPU<<<1, 5>>>();

    // La ejecución es asincrónica respecto a la CPU. Pero la ejecución de los kernels es sincrónica.
    // Primero se ejecuta el kernel de (2,10) y luego el de (1,5).

    // barrera para que todo el trabajo de la gpu termine antes de seguir...
    cudaDeviceSynchronize();

    return 0;
}


Overwriting hola.cu


nvcc es el compilador de nvidia. Funciona como cualquier compilador de C/C++ si no hay partes en CUDA. Si hay código CUDA sabe como compilarlo para su ejecución en GPU.

In [29]:
!nvcc -arch=sm_75 hola.cu -o hola

In [30]:
!./hola

Hola mundo desde la CPU!

hola mundo desde la GPU!, hilo: 0, bloque: 0
hola mundo desde la GPU!, hilo: 1, bloque: 0
hola mundo desde la GPU!, hilo: 2, bloque: 0
hola mundo desde la GPU!, hilo: 3, bloque: 0
hola mundo desde la GPU!, hilo: 4, bloque: 0
hola mundo desde la GPU!, hilo: 5, bloque: 0
hola mundo desde la GPU!, hilo: 6, bloque: 0
hola mundo desde la GPU!, hilo: 7, bloque: 0
hola mundo desde la GPU!, hilo: 8, bloque: 0
hola mundo desde la GPU!, hilo: 9, bloque: 0
hola mundo desde la GPU!, hilo: 0, bloque: 1
hola mundo desde la GPU!, hilo: 1, bloque: 1
hola mundo desde la GPU!, hilo: 2, bloque: 1
hola mundo desde la GPU!, hilo: 3, bloque: 1
hola mundo desde la GPU!, hilo: 4, bloque: 1
hola mundo desde la GPU!, hilo: 5, bloque: 1
hola mundo desde la GPU!, hilo: 6, bloque: 1
hola mundo desde la GPU!, hilo: 7, bloque: 1
hola mundo desde la GPU!, hilo: 8, bloque: 1
hola mundo desde la GPU!, hilo: 9, bloque: 1
hola mundo desde el device, hilo: 0, bloque: 0
hola mundo desde el device,

# Device Query

In [32]:
%%writefile queplacasoy.cu

#include <stdio.h>

int main(int argc, char **argv)
{
	cudaDeviceProp deviceProp;

	int deviceCount = 0;
    	cudaError_t error_id = cudaGetDeviceCount(&deviceCount);


	printf("En este nodo hay %d placas\n\n",deviceCount);
	for(int dev=0;dev<deviceCount;dev++){
	    	cudaSetDevice(dev);
    		cudaGetDeviceProperties(&deviceProp, dev);
    		printf("Hola!, yo soy [Device %d: \"%s\"], tu acelerador grafico personal\n", dev, deviceProp.name);
	}

	int dev; cudaGetDevice(&dev);
	printf("\nle asigno la device %d, que esta desocupada\n", dev);


	return 0;
}


Writing queplacasoy.cu


In [33]:
!nvcc queplacasoy.cu -o queplacasoy

In [34]:
!./queplacasoy

En este nodo hay 1 placas

Hola!, yo soy [Device 0: "Tesla T4"], tu acelerador grafico personal

le asigno la device 0, que esta desocupada


## Ejercicios
* Consultar la guía de programació y obtener mas información de la GPU.

# Hilos, bloques, grillas

In [38]:
%%writefile grillas.cu

#include <stdio.h>
#include <stdlib.h>

// kernel
__global__ void Quiensoy()
{
    printf("Soy el thread (%d,%d,%d) del bloque (%d,%d,%d) [blockDim=(%d,%d,%d),gridDim=(%d,%d,%d)] \n",
      threadIdx.x,threadIdx.y,threadIdx.z,blockIdx.x,blockIdx.y,blockIdx.z,
      blockDim.x,blockDim.y,blockDim.z,gridDim.x,gridDim.y,gridDim.z);
}

int main(int argc, char **argv)
{

	if(argc!=7) {
		printf("uso: %s ntbx ntby ntbz nbgx nbgy nbgz\n", argv[0]);

		// ejemplo 1
		printf("ejemplo 1: Quiensoy<<< 3, 2>>> == Quiensoy<<< dim3(3,1,1), dim3(2,1,1)>>>\n");
		Quiensoy<<< dim3(3,1,1), dim3(2,1,1)>>>();
		cudaDeviceSynchronize();

		// ejemplo 2
		// Indexado por bloques (2x2) y dentro de cada uno de los 4 bloques tengo (2x1) hilos.
		printf("ejemplo 2: Quiensoy<<< dim3(2,2), dim3(2,1) >>>();\n");
		dim3 nb(2,2);
		dim3 nt(2,1);
		Quiensoy<<< nb, nt >>>();
		cudaDeviceSynchronize();
	}
	else{
		dim3 nThreads_per_block;
		dim3 nBlocks_per_grid;

		nThreads_per_block.x = atoi(argv[1]);
		nThreads_per_block.y = atoi(argv[2]);
		nThreads_per_block.z = atoi(argv[3]);
		nBlocks_per_grid.x = atoi(argv[4]);
		nBlocks_per_grid.y = atoi(argv[5]);
		nBlocks_per_grid.z = atoi(argv[6]);

		// kernel
		printf("\nDesde del host lanzamos\n Quiensoy<<< dim3(%d,%d,%d), dim3(%d,%d,%d) >>>():\n\n",
		nThreads_per_block.x,nThreads_per_block.y,nThreads_per_block.z,
		nBlocks_per_grid.x,nBlocks_per_grid.y,nBlocks_per_grid.z);

		printf("Y los hilos imprimen:\n");
		Quiensoy<<< nBlocks_per_grid, nThreads_per_block >>>();
		cudaDeviceSynchronize();
	}
	return 0;
}

// La dimensión del bloque es cuántos hilos aloja. En ejemplo2 tengo (2,1,1) porque son 2 hilos en una dimensión.
// La dimensión de la grilla es cuántos bloques aloja. En ejemplo2 tengo (2,2,1).


Overwriting grillas.cu


In [36]:
!nvcc -arch=sm_75 grillas.cu -o grillas

In [37]:
!./grillas

uso: ./grillas ntbx ntby ntbz nbgx nbgy nbgz
ejemplo 1: Quiensoy<<< 3, 2>>> == Quiensoy<<< dim3(3,1,1), dim3(2,1,1)>>>
Soy el thread (0,0,0) del bloque (2,0,0) [blockDim=(2,1,1),gridDim=(3,1,1)] 
Soy el thread (1,0,0) del bloque (2,0,0) [blockDim=(2,1,1),gridDim=(3,1,1)] 
Soy el thread (0,0,0) del bloque (0,0,0) [blockDim=(2,1,1),gridDim=(3,1,1)] 
Soy el thread (1,0,0) del bloque (0,0,0) [blockDim=(2,1,1),gridDim=(3,1,1)] 
Soy el thread (0,0,0) del bloque (1,0,0) [blockDim=(2,1,1),gridDim=(3,1,1)] 
Soy el thread (1,0,0) del bloque (1,0,0) [blockDim=(2,1,1),gridDim=(3,1,1)] 
ejemplo 2: Quiensoy<<< dim3(2,2), dim3(2,1) >>>();
Soy el thread (0,0,0) del bloque (0,1,0) [blockDim=(2,1,1),gridDim=(2,2,1)] 
Soy el thread (1,0,0) del bloque (0,1,0) [blockDim=(2,1,1),gridDim=(2,2,1)] 
Soy el thread (0,0,0) del bloque (0,0,0) [blockDim=(2,1,1),gridDim=(2,2,1)] 
Soy el thread (1,0,0) del bloque (0,0,0) [blockDim=(2,1,1),gridDim=(2,2,1)] 
Soy el thread (0,0,0) del bloque (1,0,0) [blockDim=(2,1,1),g

In [ ]:
!./grillas 10 1 1 1 1 1


Desde del host lanzamos
 Quiensoy<<< dim3(10,1,1), dim3(1,1,1) >>>():

Y los hilos imprimen:
Soy el thread (0,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (1,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (2,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (3,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (4,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (5,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (6,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (7,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (8,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 
Soy el thread (9,0,0) del bloque (0,0,0) [blockDim=(10,1,1),gridDim=(1,1,1)] 


## Ejercicios:

* Experimentar con distintos números.
* ¿Qué pasa si el número de hilos por bloque es mayor a 1024?
* ¿Cual es el máximo número de bloques?

# Atrapando Errores



In [ ]:
%%writefile estefalla.cu

#include <stdio.h>
#include <cuda_runtime.h>



// CUDA error checking macro
#define CUDA_CHECK(call)                                      \
    {                                                        \
        cudaError_t err = call;                              \
        if (err != cudaSuccess) {                            \
            fprintf(stderr, "CUDA Error: %s (at %s:%d)\n",   \
                    cudaGetErrorString(err), __FILE__, __LINE__); \
            exit(EXIT_FAILURE);                              \
        }                                                    \
    }

// Failing CUDA Kernel (Out-of-Bounds Write)
__global__ void faultyKernel(int *d_array) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    // Intentional out-of-bounds access
    d_array[idx] = 42;
}

int main() {
    int *d_array;
    int size = 10 * sizeof(int);  // Allocate space for 10 integers

    // **1. Intentional Memory Allocation Failure**
    CUDA_CHECK(cudaMalloc((void**)&d_array, size * 100000000)); // Too much memory!

    // **2. Launch Kernel with Too Many Threads**
    dim3 threadsPerBlock(2048);  // Exceeds max threads per block
    dim3 numBlocks(1);

    faultyKernel<<<numBlocks, threadsPerBlock>>>(d_array);

    // **3. Check for Kernel Launch Errors**
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "Kernel launch failed: %s\n", cudaGetErrorString(err));
    }

    // **4. Synchronize to Catch Runtime Errors**
    CUDA_CHECK(cudaDeviceSynchronize());

    // Free memory (won't reach this point due to failures)
    CUDA_CHECK(cudaFree(d_array));

    return 0;
}


Overwriting estefalla.cu


In [ ]:
!nvcc -arch=sm_75 estefalla.cu -o estefalla

In [ ]:
!./estefalla

CUDA Error: out of memory (at estefalla.cu:30)


# CUDA Samples

Para aprender CUDA, este repositorio es ideal.



In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git

Cloning into 'cuda-samples'...
remote: Enumerating objects: 19507, done.
remote: Counting objects: 100% (4370/4370), done.
remote: Compressing objects: 100% (752/752), done.
remote: Total 19507 (delta 4059), reused 3618 (delta 3618), pack-reused 15137 (from 2)
Receiving objects: 100% (19507/19507), 133.52 MiB | 14.16 MiB/s, done.
Resolving deltas: 100% (17186/17186), done.
Updating files: 100% (4026/4026), done.


In [ ]:
!ls cuda-samples

bin	      Common   Makefile   Samples	      Samples_VS2019.sln
CHANGELOG.md  LICENSE  README.md  Samples_VS2017.sln  Samples_VS2022.sln


In [ ]:
!cd /content/cuda-samples/Samples/1_Utilities/deviceQuery; make clean; make

rm -f deviceQuery deviceQuery.o
rm -rf ../../../bin/x86_64/linux/release/deviceQuery
/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common -m64 --threads 0 --std=c++11 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda/bin/nvcc -ccbin g++ -m64 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=

In [ ]:
!./cuda-samples/bin/x86_64/linux/release/deviceQuery

./cuda-samples/bin/x86_64/linux/release/deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.4 / 12.5
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15095 MBytes (15828320256 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:           